# Depreciation to FERC 1 Connection

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import sqlalchemy as sa
import logging
import sys
import copy
import pathlib
import random
import warnings
from copy import deepcopy

import pudl_rmi.connect_deprish_to_eia
import pudl_rmi.make_plant_parts_eia
import pudl_rmi.connect_deprish_to_ferc1
import pudl_rmi.deprish

DistributionNotFound: The 'pudl_rmi' distribution was not found and is required by the application

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

### Prepare the inputs

In [16]:
file_path_mul = pathlib.Path().cwd().parent / 'outputs'/ 'master_unit_list.pkl.gz'
file_path_steam_ferc1 = pathlib.Path().cwd().parent /'outputs' / 'steam_ferc1.pkl.gz'
file_path_ferc1_eia = pathlib.Path().cwd().parent / 'outputs' /'ferc1_to_eia.pkl.gz'
file_path_deprish_eia = pathlib.Path().cwd().parent / 'outputs' /'deprish_to_eia.pkl.gz'

In [6]:
%%time
inputs = pudl_rmi.connect_deprish_to_ferc1.InputsCompiler(
    file_path_mul=file_path_mul,
    file_path_steam_ferc1=file_path_steam_ferc1,
    file_path_ferc1_eia=file_path_ferc1_eia,
    file_path_deprish_eia=file_path_deprish_eia
)

Reading the master unit list from /Users/christinagosnell/code/rmi-ferc1-eia/outputs/master_unit_list.pkl.gz
CPU times: user 3.39 s, sys: 801 ms, total: 4.19 s
Wall time: 4.66 s


In [7]:
inputs.prep_inputs()

### Generate the options and connections!

In [8]:
match_maker = pudl_rmi.connect_deprish_to_ferc1.MatchMaker(inputs)
matches_df = match_maker.match()

Matched plants:    49
Not connected:       12
Missing connections: 9
Matches for same_true:   46
Matches for same_diff_own:   0


/Users/christinagosnell/code/rmi-ferc1-eia/src/rmi_pudl/connect_deprish_to_ferc1.py:266: UserWarning: There are 9 missing plant records.
  warnings.warn(


Matches for same_quals:   82
Matches for one_ferc1_opt:   1
Portion of unique depreciation records:
    Matched:   91.67%
    No link:   5.00%
Connected:
same_quals       82
same_true        46
one_ferc1_opt     1
Name: match_method, dtype: int64
Connection Levels:
samezies    46
beeg        45
smol        38
Name: level_deprish, dtype: int64


In [9]:
scaler = pudl_rmi.connect_deprish_to_ferc1.Scaler(match_maker)
scaled_df = scaler.scale()
# this should be true
len(scaled_df) == len(scaler.matches_df)
logger.info(f"{len(scaled_df)} {len(scaler.matches_df)}")

Scaled via same/true: 46
Scaled via same/smol: 37
Scaled via same/beeg: 45
128 129


In [13]:
file_path_deprish = pathlib.Path().cwd().parent/ 'inputs'/'depreciation_rmi.xlsx'
sheet_name_deprish='Depreciation Studies Raw'
transformer = rmi_pudl.deprish.Transformer(
    pudl_rmi.deprish.Extractor(
        file_path=file_path_deprish,
        sheet_name=sheet_name_deprish
    ).execute())
deprish_df = transformer.execute()
deprish_asset_df = deprish.agg_to_asset(deprish_df)

Reading the depreciation data from /Users/christinagosnell/code/rmi-ferc1-eia/inputs/depreciation_rmi.xlsx


FileNotFoundError: [Errno 2] No such file or directory: '/Users/christinagosnell/code/rmi-ferc1-eia/inputs/depreciation_rmi.xlsx'

In [10]:
connected_deprish = rmi_pudl.connect_deprish_to_ferc1.rmi_output_ify(scaled_df, deprish_df)

NameError: name 'deprish_df' is not defined

In [15]:
connected_deprish

,Utility,Study Year,FERC or Digitized,FERC Utility Code,FERC Account Category,Clean Line Name,EIA Utility Code,Unit Name,ORIGINAL COST,TOTAL ACCRUALS,BOOK RESERVE,ANNUAL AMOUNT,EIA Best Match Generator,FERC Line Type,EIA Best Match Plant Name,ID,EIA Prime Mover,EIA Fuel Source,FERC Account,Plant Technology Level FERC Form 1 Matches,FERC Capacity,State,Total Nameplate Capacity (MW),Utility Ownership (%),Net Generation (MWh),Net Fuel Costs ($),Net Total Production Costs,Net Fuel Costs ($/MWh),Net Operating Costs ($/MWh),Net Capacity Factor (%)
0,Public Service Company of Colorado,2018,PUC,145,Steam,Cherokee 4,15466,4,14894894.00,NaN,NaN,992998.0,4,plant_unit,Cherokee,469,ST,gas,312.1,Natural Gas Steam Turbine,505.8,CO,380.8,1.0,1090864.0,4.444992e+07,13382813.0,41.473631,11.883042,0.254177
1,Public Service Company of Colorado,2018,PUC,145,Steam,Cherokee 4,15466,4,59493607.00,2.015681e+07,3.933680e+07,3422548.0,4,plant_unit,Cherokee,469,ST,gas,312,Natural Gas Steam Turbine,505.8,CO,380.8,1.0,1090864.0,4.444992e+07,13382813.0,41.473631,11.883042,0.254177
2,Public Service Company of Colorado,2018,PUC,145,Steam,Cherokee 4,15466,4,20913647.00,6.332785e+06,1.458086e+07,1112522.0,4,plant_unit,Cherokee,469,ST,gas,314,Natural Gas Steam Turbine,505.8,CO,380.8,1.0,1090864.0,4.444992e+07,13382813.0,41.473631,11.883042,0.254177
3,Public Service Company of Colorado,2018,PUC,145,Steam,Cherokee 4,15466,4,9139690.00,2.968629e+06,6.171061e+06,504337.0,4,plant_unit,Cherokee,469,ST,gas,311,Natural Gas Steam Turbine,505.8,CO,380.8,1.0,1090864.0,4.444992e+07,13382813.0,41.473631,11.883042,0.254177
4,Public Service Company of Colorado,2018,PUC,145,Steam,Cherokee 4,15466,4,7307248.00,2.139609e+06,5.167639e+06,374584.0,4,plant_unit,Cherokee,469,ST,gas,315,Natural Gas Steam Turbine,505.8,CO,380.8,1.0,1090864.0,4.444992e+07,13382813.0,41.473631,11.883042,0.254177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,"Duke Energy Progress, Inc.",2016,PUC,17,NaN,Blewett,3046,<NA>,8275323.29,6.651482e+06,5.016724e+06,184010.0,NaN,plant,Blewett,2707,NaN,oil,332,NaN,70.0,NaN,94.6,1.0,0.0,0.000000e+00,351445.0,-2383.780822,-4814.315068,-0.000119
552,"Duke Energy Progress, Inc.",2016,PUC,17,NaN,Blewett,3046,<NA>,7082284.69,1.042055e+07,-4.345250e+05,342811.0,NaN,plant,Blewett,2707,NaN,oil,333,NaN,70.0,NaN,94.6,1.0,0.0,0.000000e+00,351445.0,-2383.780822,-4814.315068,-0.000119
553,"Duke Energy Progress, Inc.",2016,PUC,17,NaN,Blewett,3046,<NA>,7366884.27,9.820530e+06,5.667770e+05,280535.0,NaN,plant,Blewett,2707,NaN,oil,334,NaN,70.0,NaN,94.6,1.0,0.0,0.000000e+00,351445.0,-2383.780822,-4814.315068,-0.000119
554,"Duke Energy Progress, Inc.",2016,PUC,17,NaN,Blewett,3046,<NA>,1272690.93,1.449814e+06,3.446800e+05,48005.0,NaN,plant,Blewett,2707,NaN,oil,335,NaN,70.0,NaN,94.6,1.0,0.0,0.000000e+00,351445.0,-2383.780822,-4814.315068,-0.000119


In [51]:
file_path_dc = pathlib.Path().cwd().parent /'outputs' /'deprish_w_common.csv.gz'
deprish_df.to_csv(file_path_dc)

In [50]:
file_path_cd = pathlib.Path().cwd().parent /'outputs' /'deprish_to_ferc.csv.gz'
connected_deprish.to_csv()

In [52]:
file_path_da = pathlib.Path().cwd().parent /'outputs' /'deprish_w_common_by_asset.csv.gz'
deprish_asset_df.to_csv(file_path_da)

In [12]:
file_path_scaled = pathlib.Path().cwd().parent /'outputs' /'ferc_to_deprish_scaled.csv.gz'
scaled_df.to_csv(file_path_scaled)

In [73]:
bad_hombres = deprish_asset_df[~deprish_asset_df.plant_balance_diff_check.between(.95,1.5)].sort_values(by=['utility_id_pudl']).reset_index()
# file_path_deprish_bad = pathlib.Path().cwd().parent / 'outputs' /'depreciation_scaled_bad.csv'
# bad_hombres.to_csv(file_path_deprish_bad)
# bad_hombres.loc[:int(len(bad_hombres)/2)]

In [ ]:
# we should figure out a cleaner/automatic way to do this....
# but for now I'm just enumerating the columns I want to keep
cols_to_keep = [
    "plant_part_deprish", "plant_part_ferc1", "record_id_eia_deprish", 
    "record_id_eia_ferc1", "plant_part_name", "plant_name_match", 
    "fraction_owned_deprish", "fraction_owned_ferc1", 
    "record_count_deprish", "record_count_ferc1", "plant_part_name_match", 
    "state", "utility_name_ferc1_deprish", "plant_id_pudl", 
    "utility_id_ferc1_deprish", "true_gran_name_match", "report_date", 
    "plant_name_new_deprish", "report_year_deprish", "ownership_deprish", 
    "plant_name_eia_deprish", "plant_id_eia_deprish", 
    "generator_id_deprish", "unit_id_pudl_deprish", 
    "prime_mover_code_deprish", "energy_source_code_1_deprish", 
    "technology_description_deprish", "ferc_acct_name_deprish", 
    "utility_id_eia_deprish", "utility_id_pudl", "true_gran_deprish", 
    "appro_part_label_deprish", "appro_record_id_eia_deprish", 
    "ownership_dupe_deprish", "total_fuel_cost_deprish", 
    "net_generation_mwh_deprish", "capacity_mw_deprish", 
    "record_id_ferc1", "utility_id_ferc1_ferc1", 
    "utility_name_ferc1_ferc1", "plant_id_ferc1", "plant_name_ferc1", 
    "asset_retirement_cost", "avg_num_employees", "capacity_factor", 
    "capacity_mw_ferc1", "capex_equipment", "capex_land", "capex_per_mw", 
    "capex_structures", "capex_total", "construction_type", 
    "construction_year", "installation_year", "net_generation_mwh_ferc1", 
    "not_water_limited_capacity_mw", "opex_allowances", "opex_boiler", 
    "opex_coolants", "opex_electric", "opex_engineering", "opex_fuel", 
    "fuel_cost_per_mwh", "opex_misc_power", "opex_misc_steam", 
    "opex_nonfuel_per_mwh", "opex_operations", "opex_per_mwh", 
    "opex_plants", "opex_production_total", "opex_rents", "opex_steam", 
    "opex_steam_other", "opex_structures", "opex_transfer", "peak_demand_mw", 
    "plant_capability_mw", "plant_hours_connected_while_generating", 
    "plant_type", "water_limited_capacity_mw", "total_fuel_cost_ferc1", 
    "total_mmbtu", "fuel_type_code_pudl", "fuel_cost_per_mmbtu", 
    "heat_rate_mmbtu_mwh", "plant_id_report_year", 
    "plant_id_report_year_util_id", "opex_nonfuel", "opex_nonfuel_own_frac", 
    "net_generation_mwh_ferc1_own_frac"]

In [ ]:
# we want to grab the depreciation data so the scaled output can have
# ALL OF THE DATA
file_path_deprish = pathlib.Path().cwd().parent/'depreciation_rmi.xlsx'
sheet_name_deprish=0
deprish_df = (
    deprish.Transformer(
        deprish.Extractor(file_path=file_path_deprish,
                          sheet_name=sheet_name_deprish).execute())
    .execute()
    .dropna(subset=connect_deprish_to_eia.RESTRICT_MATCH_COLS)
)

matches_df = pd.merge(
    deprish_df,
    scaler.matches_df[cols_to_keep],
    how='outer'
)

In [ ]:
file_path_deprish_ferc = pathlib.Path().cwd().parent / 'outputs' /'deprish_to_ferc.csv.gz'
matches_df.to_csv(file_path_deprish_ferc)

In [ ]:
# testing the scaling
same_smol = scaler.split_ferc1_data_cols()
same_beeg = scaler.agg_ferc_data_cols()

first_cols = ['plant_part_deprish', 'plant_part_ferc1',
              'record_id_eia_deprish', 'record_id_eia_ferc1',
              'plant_name', 'plant_name_match', 'plant_name_ferc1', 'plant_name_new_ferc1',
              'fraction_owned_deprish', 'fraction_owned_ferc1',
              'record_count_deprish', 'record_count_ferc1',
              'record_count_matches_deprish'
              ]
def reorder_cols(df, first_cols):
    # reorder cols so they are easier to see, maybe remove later
    df = df[
        [x for x in first_cols if x in df.columns] 
        + [x for x in df.columns if x not in first_cols]]
    return df
matches_df = reorder_cols(matches_df, first_cols)
same_smol = reorder_cols(same_smol, first_cols)
same_beeg = reorder_cols(same_beeg, first_cols)

In [ ]:
mergey = pd.merge(
    inputs.connects_ferc1_eia.assign(report_year=lambda x: x.report_date.dt.year),
    pudl_out.fbp_ferc1(),
    on=['report_year','utility_id_ferc1','plant_id_pudl','plant_name_ferc1'],
    validate='m:1'
)[['report_year','utility_id_ferc1','plant_id_pudl','plant_name_ferc1','energy_source_code_1','primary_fuel_by_mmbtu']]

esc_to_tech = {}
for tech, escs in pc.fuel_type_eia923_gen_fuel_simple_map.items():
    for esc in escs:
        esc_to_tech[esc] = tech
        
mergey["tech_type_eia"] = mergey["energy_source_code_1"].str.lower().map(esc_to_tech)

len(mergey[
    (mergey['tech_type_eia'] == mergey['primary_fuel_by_mmbtu'])
    & (mergey.tech_type_eia.notnull())
])/len(mergey[(mergey.tech_type_eia.notnull())])

#len(pudl_out.plants_small_ferc1()[pudl_out.plants_small_ferc1().fuel_type.notnull()])/len(pudl_out.plants_small_ferc1())